# Part 1

## All import

In [ ]:
#!pip install datasets transformers

In [ ]:
import transformers

print(transformers.__version__)

4.12.5


In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from tqdm.auto import tqdm
import collections
import numpy as np
import pandas as pd

## Getting the dataset

In [ ]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
datasets = load_dataset("squad_v2")
split_datasets = load_dataset("squad_v2", split=['train[:5%]', 'validation[:10%]'])
datasets['train'] = split_datasets[0]
datasets['validation'] = split_datasets[1]

model_checkpoint = "bert-base-uncased"
batch_size = 10

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 6516
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1187
    })
})

## Our trainer

In [ ]:
class MyTrainer:
  def __init__(self, model, model_checkpoint: str, datasets, batchsize: int):
    model_name = model_checkpoint.split("/")[-1]
    args = TrainingArguments(
      f"{model_name}-finetuned-squad",
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=3,
      weight_decay=0.01,
      push_to_hub=False,
      #disable_tqdm=True
    )
    self.train_dataset = self.preprocess(model_checkpoint, datasets)
    data_collator = default_data_collator
    self.trainer = Trainer(
      model,
      args,
      train_dataset=self.train_dataset["train"],
      eval_dataset=self.train_dataset["validation"],
      data_collator=data_collator,
      tokenizer=self.tokenizer,
    )

  def preprocess(self, model_checkpoint: str, datasets):
    self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    self.pad_on_right = self.tokenizer.padding_side == "right"
    self.max_length = 384 # The maximum length of a feature (question and context)
    self.doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    return datasets.map(self.prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  def prepare_train_features(self, examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = self.tokenizer(
        examples["question" if self.pad_on_right else "context"],
        examples["context" if self.pad_on_right else "question"],
        truncation="only_second" if self.pad_on_right else "only_first",
        max_length=self.max_length,
        stride=self.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(self.tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if self.pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if self.pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

## Our evaluator

In [ ]:
class MyEvaluator:
  def __init__(self, tokenizer):
    self.metric = load_metric("squad_v2")
    self.max_length = 384 # The maximum length of a feature (question and context)
    self.doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    self.tokenizer = tokenizer
    self.pad_on_right = self.tokenizer.padding_side == "right"

  def predict(self, datasets, trainer):
    ## Getting all features and predictions
    validation_features = datasets.map(
      self.prepare_validation_features,
      batched=True,
      remove_columns=datasets.column_names
    )

    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    ## Post process everything
    final_predictions = self.postprocess_qa_predictions(datasets, validation_features, raw_predictions.predictions)
    return [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

  def evaluate(self, datasets, trainer):
    ## Getting all features and predictions
    validation_features = datasets["validation"].map(
      self.prepare_validation_features,
      batched=True,
      remove_columns=datasets["validation"].column_names
    )
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    ## Post process everything
    final_predictions = self.postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
    self.formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
    self.references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
    return self.metric.compute(predictions=self.formatted_predictions, references=self.references)

  def prepare_validation_features(self, examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = self.tokenizer(
        examples["question" if self.pad_on_right else "context"],
        examples["context" if self.pad_on_right else "question"],
        truncation="only_second" if self.pad_on_right else "only_first",
        max_length=self.max_length,
        stride=self.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if self.pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

  def postprocess_qa_predictions(self, examples, features, raw_predictions, n_best_size: int = 20, max_answer_length: int = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    #print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(examples):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(self.tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

## Our model

In [ ]:
class MyModel:
  def __init__(self, model_checkpoint: str, datasets, batchsize: int):
    # For future load
    self.model_checkpoint = model_checkpoint
    self.datasets = datasets
    self.batchsize = batchsize

    # Our model
    self.model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    # Our trainer
    self.trainer = MyTrainer(self.model, model_checkpoint, datasets, batchsize)
    # Our evaluator
    self.evaluator = MyEvaluator(self.trainer.tokenizer)

  def train(self):
    self.trainer.trainer.train()
  def save(self):
    self.model.save_pretrained("trained_model")
    !zip -r trained_model.zip trained_model
  def evaluate(self, datasets):
    return self.evaluator.evaluate(datasets, self.trainer.trainer)
  def predict(self, datasets):
    return self.evaluator.predict(datasets, self.trainer.trainer)
  def load(self):
    !unzip trained_model.zip
    self.model = AutoModelForQuestionAnswering.from_pretrained("trained_model")
    self.trainer = MyTrainer(self.model, self.model_checkpoint, self.datasets, self.batchsize)
    self.evaluator = MyEvaluator(self.trainer.tokenizer)

## Training and Evaluating

In [ ]:
model = MyModel(model_checkpoint, datasets, batch_size)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-7ea86b79305bfab9.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
#model.train()
#model.save()

In [ ]:
model.load()

Archive:  trained_model.zip
   creating: trained_model/
  inflating: trained_model/config.json  
  inflating: trained_model/pytorch_model.bin  


Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-7ea86b79305bfab9.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-574cb4c285245157.arrow


In [ ]:
metrics = model.evaluate(datasets)
metrics

  0%|          | 0/2 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 1194
  Batch size = 10


{'HasAns_exact': 58.22147651006711,
 'HasAns_f1': 68.98865295699447,
 'HasAns_total': 596,
 'NoAns_exact': 0.8460236886632826,
 'NoAns_f1': 0.8460236886632826,
 'NoAns_total': 591,
 'best_exact': 50.2106149957877,
 'best_exact_thresh': 0.0,
 'best_f1': 50.2106149957877,
 'best_f1_thresh': 0.0,
 'exact': 29.65459140690817,
 'f1': 35.06085691859201,
 'total': 1187}

## Using mvonwyl model

In [ ]:
mvonwyl_model = MyModel("mvonwyl/distilbert-base-uncased-finetuned-squad2", datasets, batch_size)

In [ ]:
mvonwyl_metrics = mvonwyl_model.evaluate(datasets)
mvonwyl_metrics

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 12134
  Batch size = 10


Post-processing 11873 example predictions split into 12134 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.41363022941971,
 'HasAns_f1': 73.5008343719444,
 'HasAns_total': 5928,
 'NoAns_exact': 63.33052985702271,
 'NoAns_f1': 63.33052985702271,
 'NoAns_total': 5945,
 'best_exact': 64.86987282068559,
 'best_exact_thresh': 0.0,
 'best_f1': 68.40840109129043,
 'best_f1_thresh': 0.0,
 'exact': 64.86987282068559,
 'f1': 68.4084010912902,
 'total': 11873}

## Showing some bad predictions

In [ ]:
predictions = mvonwyl_model.evaluator.formatted_predictions
references = mvonwyl_model.evaluator.references

In [ ]:
def my_pretty_print(ct: str):
  i = 0
  l = len(ct)
  while i*150 < l:
    print(ct[i*150:(i+1)*150])
    i+=1

In [ ]:
tot = 0
for prediction, reference in zip(predictions, references):
  pred = prediction['prediction_text']
  ref = reference['answers']['text']
  if ref != [] and pred not in ref:
    print("Prediction: ",pred)
    print("Reference: ",ref)
    print('Question : ', [data['question'] for data in datasets['validation'] if data['id'] == prediction['id']][0])
    context = [data['context'] for data in datasets['validation'] if data['id'] == prediction['id']][0] + "\n"
    print('Context : ')
    my_pretty_print(context)
    tot += 1
  if tot == 3:
    break


Prediction:  Norseman, Viking"
Reference:  ['Viking', 'Norseman, Viking', 'Norseman, Viking']
Question :  What is the original meaning of the word Norman?
Context : 
The English name "Normans" comes from the French words Normans/Normanz, plural of Normant, modern French normand, which is itself borrowed from Old Lo
w Franconian Nortmann "Northman" or directly from Old Norse Norðmaðr, Latinized variously as Nortmannus, Normannus, or Nordmannus (recorded in Medieva
l Latin, 9th century) to mean "Norseman, Viking".

Prediction:  King Charles III of West Francia
Reference:  ['King Charles III', 'King Charles III', 'King Charles III']
Question :  Who did Rollo sign the treaty of Saint-Clair-sur-Epte with?
Context : 
In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encam
pments that included local women and personal property. The Duchy of Normandy, which began in 911 as a fiefdom, was established 

# Part 2

## Pip install and imports

In [ ]:
"""%%capture
!pip install -U sentence-transformers
!pip install beir
!pip install numpy
!pip install datasets"""

In [ ]:
import numpy as np
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm_notebook

## All functions

In [ ]:
def create_context_dataset() -> [str]:
    '''
    Returns a list of 10000 contexts, around 1200 from Squadv2 and others from DBpedia
    '''

    #Load DBpedia
    dataset = "DBpedia-Entity"
    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset.lower())
    data_path = util.download_and_unzip(url, "datasets")
    corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

    #Load Squadv2 and get the validation set
    datasets = load_dataset("squad_v2")
    val = datasets['validation']

    context_list = []

    #Get all different contexts from Squadv2 validation set
    for elt in val:
        if elt['context'] not in context_list:
            context_list.append(elt['context'])

    #Fill the context list with contexts from DBpedia up to a total of 10000 (contexts must contain at least 50 words)
    for e in corpus:
        context = corpus.get(e).get("text")
        if len(context.split()) > 50:
            context_list.append(context)
        if len(context_list) >= 10000:
            break

    return context_list

In [ ]:
def context_to_vector(context_list: [str], transformer) -> [[float]]:
    '''
    Returns a list of vectors that contains vectorization of all contexts
    '''

    #Use the transformer on every context
    result = []
    for context in context_list:
      result.append(transformer.encode(context))
    return result

In [ ]:
def get_sorted_similarities(query: str, model, encoded: [[float]], k: int) -> [(float, int)]:
    '''
    Returns the top k results from a sorted list of similiraty between the query and all encoded contexts
        Parameters:
            query (str): Contains a question
            model: The transformer
            encoded ([[int]]): List of vectorized contexts
            k (int): Number of results returned
    '''

    #Get vector from query
    query_embedding = model.encode(query)

    #Calculate similarity between query and all contexts
    similarities = [(
        np.dot(context_vector, query_embedding)/(np.linalg.norm(context_vector)*np.linalg.norm(query_embedding)),
        i
      ) for i, context_vector in enumerate(encoded)]

    #Sort the list
    similarities.sort(reverse=True)

    return similarities[:k]

In [ ]:
def change_tmp_datasets(data, question : str, context: str):
  data['context'] = context
  data['question'] = question
  return data

In [ ]:
def MRR(context_list: [str], datasets) -> float:
  """
    Compute mean reciprocal rank
  """
    mrr = []
    tmp_datasets = datasets['train'].select(range(1))
    for elt in datasets['validation'].select(range(200)):
        contexts = get_sorted_similarities(elt['question'], transformer, encoded_contexts, 20)
        y_true = elt['answers']['text']
        for i in range(len(contexts)):
            tmp_datasets = tmp_datasets.map(lambda x: change_tmp_datasets(x, elt['question'], context_list[contexts[i][1]]))
            y_pred = model.predict(tmp_datasets)[0]['prediction_text']
            if y_pred in y_true:
                break
        mrr.append(1/(i + 1))
    return np.array(mrr).mean()

## Getting similarites

In [ ]:
context_list = create_context_dataset()

datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

  0%|          | 0/4635922 [00:00<?, ?it/s]

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#Create the transformer that will create vectors from texts
transformer = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [ ]:
#Create vectorized context list
encoded_contexts = context_to_vector(context_list, transformer)

In [ ]:
%%capture

transformers.logging.set_verbosity_error()
mrr = MRR(context_list, datasets)
transformers.logging.set_verbosity_info()

In [ ]:
print(mrr)

0.18183916083916082


## Global predict

In [ ]:
def Global_predict(question: str) -> [str]:
  """
    Return a list of answers to a question
  """
    tmp_datasets = datasets['train'].select(range(1))
    contexts = get_sorted_similarities(question, transformer, encoded_contexts, 5)
    result = []
    for i in range(len(contexts)):
      tmp_datasets = tmp_datasets.map(lambda x: change_tmp_datasets(x, question, context_list[contexts[i][1]]))
      y_pred = model.predict(tmp_datasets)[0]['prediction_text']
      result.append(y_pred)
    return result


In [ ]:
%%capture
transformers.logging.set_verbosity_error()
answers = Global_predict("Who were the people who lived Normandy ?")
transformers.logging.set_verbosity_info()

In [ ]:
for answer in answers:
  print(answer)

The Normans
Norman: Normaundie, from Old French Normanz, plural of Normant, originally from the word for "northman
the Anglo-Saxons
Franks
The Norman dynasty
